# Azure AI Agents with Model Context Protocol (MCP) Support

हे नोटबुक Azure AI Agents चा वापर करून Model Context Protocol (MCP) साधनांसह एक बुद्धिमान एजंट तयार करण्याची प्रक्रिया दाखवते, जो बाह्य MCP सर्व्हर्सचा उपयोग करून सुधारित क्षमता प्राप्त करू शकतो.


## आवश्यक NuGet पॅकेजेस स्थापित करा

सर्वप्रथम, आपल्याला Azure AI Agents Persistent पॅकेज स्थापित करावे लागेल, जे Azure AI Agents सोबत काम करण्यासाठी मुख्य कार्यक्षमता प्रदान करते.


## कीलेस प्रमाणीकरणाचे फायदे

ही नोटबुक **कीलेस प्रमाणीकरण** प्रदर्शित करते, ज्यामुळे अनेक फायदे मिळतात:
- ✅ **API कीज व्यवस्थापित करण्याची गरज नाही** - Azure ओळख-आधारित प्रमाणीकरणाचा वापर करते
- ✅ **वाढीव सुरक्षा** - कोड किंवा कॉन्फिगरेशनमध्ये कोणतेही गुपित साठवले जात नाही
- ✅ **स्वयंचलित क्रेडेन्शियल रोटेशन** - Azure क्रेडेन्शियल जीवनचक्र हाताळते
- ✅ **भूमिका-आधारित प्रवेश** - सूक्ष्म परवानग्यांसाठी Azure RBAC चा वापर करते

`DefaultAzureCredential` आपोआप सर्वोत्तम उपलब्ध क्रेडेन्शियल स्रोत वापरेल:
1. व्यवस्थापित ओळख (Azure मध्ये चालवताना)
2. Azure CLI क्रेडेन्शियल्स (विकासादरम्यान)
3. Visual Studio क्रेडेन्शियल्स
4. पर्यावरणीय चल (जर कॉन्फिगर केले असेल तर)


In [1]:
#r "nuget: Azure.AI.Agents.Persistent, 1.1.0-beta.4"

Installed Packages Azure.AI.Agents.Persistent, 1.1.0-beta.4

In [2]:
#r "nuget: Azure.Identity, 1.14.2"

Installed Packages Azure.Identity, 1.14.2

## आवश्यक नेमस्पेस आयात करा

Azure AI Agents आणि Azure Identity साठी आवश्यक नेमस्पेस आयात करा.


In [3]:
using Azure.AI.Agents.Persistent;
using Azure.Identity;

## Azure AI Agent Client (कीलेस प्रमाणीकरण) कॉन्फिगर करा

कॉन्फिगरेशन व्हेरिएबल्स सेट करा आणि **कीलेस प्रमाणीकरण** वापरून PersistentAgentsClient तयार करा:
- **projectEndpoint**: Azure AI Foundry प्रकल्पाचा एंडपॉइंट
- **modelDeploymentName**: तैनात केलेल्या AI मॉडेलचे नाव (GPT-4.1 नॅनो)
- **mcpServerUrl**: MCP सर्व्हरचा URL (Microsoft Learn API)
- **mcpServerLabel**: MCP सर्व्हर ओळखण्यासाठी लेबल
- **DefaultAzureCredential**: व्यवस्थापित ओळख, Azure CLI किंवा इतर क्रेडेन्शियल स्रोत वापरते (API की आवश्यक नाहीत)


In [ ]:
var projectEndpoint = "Your Azure AI Foundry Project Endpoint";
var modelDeploymentName = "Your Azure OpenAI Model Deployment Name";
var mcpServerUrl = "https://learn.microsoft.com/api/mcp";
var mcpServerLabel = "mslearn";
PersistentAgentsClient agentClient = new(projectEndpoint, new DefaultAzureCredential());

## MCP साधन परिभाषा तयार करा

Microsoft Learn MCP सर्व्हरशी जोडणारी MCP साधन परिभाषा तयार करा. यामुळे एजंटला Microsoft Learn सामग्री आणि दस्तऐवजांमध्ये प्रवेश मिळेल.


In [ ]:
MCPToolDefinition mcpTool = new(mcpServerLabel, mcpServerUrl);

## एआय एजंट तयार करा

निर्दिष्ट मॉडेल आणि MCP साधनांसह एक स्थिर एआय एजंट तयार करा. एजंट खालीलप्रमाणे कॉन्फिगर केलेला आहे:
- GPT-4.1 नॅनो मॉडेल
- सहाय्यासाठी MCP साधने वापरण्याच्या सूचना
- Microsoft Learn MCP सर्व्हरवर प्रवेश


In [ ]:
PersistentAgent agent = await agentClient.Administration.CreateAgentAsync(
   model: modelDeploymentName,
   name: "my-learn-agent",
   instructions: "You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
   tools: [mcpTool]
   );

## थ्रेड तयार करा आणि संदेश पाठवा

संवाद थ्रेड तयार करा आणि वापरकर्त्याला Azure OpenAI आणि OpenAI यांच्यातील फरकाबद्दल विचारणारा संदेश पाठवा. हे एजंटच्या MCP साधनांचा वापर करून अचूक माहिती प्रदान करण्याच्या क्षमतेची चाचणी घेईल.


In [7]:
PersistentAgentThread thread = await agentClient.Threads.CreateThreadAsync();

// Create message to thread
PersistentThreadMessage message = await agentClient.Messages.CreateMessageAsync(
    thread.Id,
    MessageRole.User,
    "What's difference between Azure OpenAI and OpenAI?");

## MCP साधन संसाधने कॉन्फिगर करा (कीलेस)

MCP साधन संसाधने सेट अप करा. खरोखरच कीलेस दृष्टिकोनासाठी, जर MCP सर्व्हर Azure ओळख-आधारित प्रमाणीकरणाला समर्थन देत असेल तर तुम्ही सानुकूल हेडर्स काढून टाकू शकता. खालील उदाहरणात, जर आवश्यक असेल तर हेडर्स कसे जोडायचे हे दाखवले आहे, परंतु कीलेस परिस्थितीत, हे आवश्यक नसू शकते.


In [ ]:
// Option 1: Completely keyless (if MCP server supports Azure identity)
MCPToolResource mcpToolResource = new(mcpServerLabel);
ToolResources toolResources = mcpToolResource.ToToolResources();

// Option 2: With custom headers (if still needed for specific MCP servers)
// MCPToolResource mcpToolResource = new(mcpServerLabel);
// mcpToolResource.UpdateHeader("Authorization", "Bearer <your-token>");
// ToolResources toolResources = mcpToolResource.ToToolResources();

## एजंट रन सुरू करा

वापरकर्त्याचा संदेश प्रक्रिया करण्यासाठी रन तयार करा आणि सुरू करा. एजंट कॉन्फिगर केलेल्या MCP साधने आणि संसाधने वापरून प्रतिसाद तयार करेल.


In [9]:
ThreadRun run = await agentClient.Runs.CreateRunAsync(thread, agent, toolResources);

## रन मॉनिटर करा आणि टूल मंजुरी हाताळा (कीलेस)

एजंट रन स्थिती मॉनिटर करा आणि आवश्यक टूल मंजुरी हाताळा. या लूपमध्ये:
1. रन पूर्ण होईपर्यंत किंवा कृती आवश्यक होईपर्यंत प्रतीक्षा करते
2. आवश्यक असल्यास MCP टूल कॉल्स आपोआप मंजूर करते
3. कीलेस प्रमाणीकरणासाठी, MCP सर्व्हर Azure ओळख समर्थन करत असल्यास हेडर्स आवश्यक नसतील


In [ ]:
while (run.Status == RunStatus.Queued || run.Status == RunStatus.InProgress || run.Status == RunStatus.RequiresAction)
{
    await Task.Delay(TimeSpan.FromMilliseconds(1000));
    run = await agentClient.Runs.GetRunAsync(thread.Id, run.Id);

    if (run.Status == RunStatus.RequiresAction && run.RequiredAction is SubmitToolApprovalAction toolApprovalAction)
    {
        var toolApprovals = new List<ToolApproval>();
        foreach (var toolCall in toolApprovalAction.SubmitToolApproval.ToolCalls)
        {
            if (toolCall is RequiredMcpToolCall mcpToolCall)
            {
                Console.WriteLine($"Approving MCP tool call: {mcpToolCall.Name}");
                
                // Option 1: Keyless approval (no headers needed)
                toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true));
                
                // Option 2: With headers (if required by specific MCP server)
                // toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true)
                // {
                //     Headers = { ["Authorization"] = "Bearer <your-token>" }
                // });
            }
        }

        if (toolApprovals.Count > 0)
        {
            run = await agentClient.Runs.SubmitToolOutputsToRunAsync(thread.Id, run.Id, toolApprovals: toolApprovals);
        }
    }
}

Approving MCP tool call: microsoft_docs_search


## संभाषणाचे परिणाम दर्शवा

थ्रेडमधील सर्व संदेश पुनर्प्राप्त करा आणि दाखवा, ज्यामध्ये वापरकर्त्याचा प्रश्न आणि एजंटचे उत्तर दोन्ही समाविष्ट आहेत. संदेश कालक्रमानुसार टाइमस्टॅम्प आणि भूमिकेचे निर्देशकांसह प्रदर्शित केले जातात.


In [12]:
using Azure;

AsyncPageable<PersistentThreadMessage> messages = agentClient.Messages.GetMessagesAsync(
    threadId: thread.Id,
    order: ListSortOrder.Ascending
);

await foreach (PersistentThreadMessage threadMessage in messages)
{
    Console.Write($"{threadMessage.CreatedAt:yyyy-MM-dd HH:mm:ss} - {threadMessage.Role,10}: ");
    foreach (MessageContent contentItem in threadMessage.ContentItems)
    {
        if (contentItem is MessageTextContent textItem)
        {
            Console.Write(textItem.Text);
        }
        else if (contentItem is MessageImageFileContent imageFileItem)
        {
            Console.Write($"<image from ID: {imageFileItem.FileId}>");
        }
        Console.WriteLine();
    }
}

2025-07-16 06:39:43 -       user: What's difference between Azure OpenAI and OpenAI?
2025-07-16 06:39:51 -  assistant: The main difference between Azure OpenAI and OpenAI lies in their deployment, management, and integration options:

1. **Azure OpenAI**:
   - A cloud service offered through Microsoft Azure.
   - Provides access to OpenAI models with additional enterprise features like security, compliance, and scale.
   - Allows integration with other Azure services, enabling seamless use within existing Azure-based solutions.
   - Offers managed deployment, monitoring, and support within the Azure ecosystem.
   - Suitable for organizations looking for enterprise-grade security, compliance, and regional availability.

2. **OpenAI (OpenAI API)**:
   - A standalone API service provided directly by OpenAI.
   - Accessible via the OpenAI platform without the need for Azure.
   - Focused on providing GPT models, DALL-E, etc., primarily for developers and researchers.
   - Suitable for indi


---

**अस्वीकरण**:  
हा दस्तऐवज AI भाषांतर सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) वापरून भाषांतरित करण्यात आला आहे. आम्ही अचूकतेसाठी प्रयत्नशील असलो तरी कृपया लक्षात ठेवा की स्वयंचलित भाषांतरांमध्ये त्रुटी किंवा अचूकतेचा अभाव असू शकतो. मूळ भाषेतील दस्तऐवज हा अधिकृत स्रोत मानला जावा. महत्त्वाच्या माहितीसाठी व्यावसायिक मानवी भाषांतराची शिफारस केली जाते. या भाषांतराचा वापर करून निर्माण होणाऱ्या कोणत्याही गैरसमज किंवा चुकीच्या अर्थासाठी आम्ही जबाबदार राहणार नाही.
